### Notebook para la creacion de features basicos, por ejemplo, count de eventos con pivot tables. Al finalizar genera un archivo csv con nombre "features_basicos.csv"

In [1]:
import pandas as pd
import numpy as np
import datetime
from datetime import datetime, timedelta
import utils
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=RuntimeWarning)

In [2]:
events = utils.get_events()
events.head()

,timestamp,event,person,url,sku,model,condition,storage,color,skus,...,city,region,country,device_type,screen_resolution,operating_system_version,browser_version,month,day,hora
0,2018-05-18 00:11:59,viewed product,4886f805,NaN,9288.0,Samsung Galaxy J7 Prime,Excelente,32GB,Dourado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,18,0
1,2018-05-18 00:11:27,viewed product,ad93850f,NaN,304.0,iPhone 5s,Muito Bom,32GB,Cinza espacial,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,18,0
2,2018-05-18 00:11:16,viewed product,0297fc1e,NaN,6888.0,iPhone 6S,Muito Bom,64GB,Prateado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,18,0
3,2018-05-18 00:11:14,viewed product,2d681dd8,NaN,11890.0,iPhone 7,Bom,128GB,Vermelho,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,18,0
4,2018-05-18 00:11:09,viewed product,cccea85e,NaN,7517.0,LG G4 H818P,Excelente,32GB,Branco,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,18,0


In [3]:
df_events  = utils.get_features_iniciales(events,'event')
df_channel = utils.get_features_iniciales(events,'channel')

In [4]:
df_events.head()

event,person,ad campaign hit,brand listing,checkout,conversion,generic listing,lead,search engine hit,searched products,staticpage,viewed product,visited site
0,0008ed71,0,0,3,0,1,0,0,0,0,0,2
1,00091926,15,25,2,0,0,0,0,0,0,372,34
2,00091a7a,1,5,0,0,0,0,0,0,0,3,1
3,000ba417,1,24,6,1,14,0,1,0,0,153,6
4,000c79fe,1,0,1,0,1,0,1,9,0,3,1


In [5]:
df_channel = df_channel.drop(df_channel.columns[1], axis=1)
df_channel = df_channel.fillna(0)
df_channel.head()

channel,person,Direct,Email,Organic,Paid,Referral,Social
0,0008ed71,2,0,0,0,0,0
1,00091926,1,0,25,0,0,0
2,00091a7a,0,0,0,0,0,0
3,000ba417,0,0,6,0,0,0
4,000c79fe,0,0,0,0,0,0


In [36]:
df_basicos = utils.add_features(df_channel,df_events,on='person',how='right').fillna(0)

In [22]:
events.timestamp = pd.to_datetime(events['timestamp'],format='%Y-%m-%d %H:%M:%S')
events['month'] = events.timestamp.dt.month
events['day'] = events.timestamp.dt.day
events['hour'] = events.timestamp.dt.hour

In [27]:
def timestamp_score(hora, dia, mes):
    if (mes < 3):
        return 1
    if (mes < 5):
        return 2
    return (dia/2)

    
time = events[['person', 'timestamp', 'event', 'hour', 'day', 'month']]
time['time_score'] = time[['hour','day','month']].apply(lambda x: timestamp_score(x['hour'], x['day'], x['month']), axis=1)

/home/camila/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


In [28]:
time = pd.get_dummies(time, columns=['event'])

In [29]:
time[['event_ad campaign hit', 'event_brand listing', 'event_checkout',
       'event_conversion', 'event_generic listing', 'event_lead',
       'event_search engine hit', 'event_searched products',
       'event_staticpage', 'event_viewed product', 'event_visited site']] = time[['event_ad campaign hit', 'event_brand listing', 'event_checkout',
       'event_conversion', 'event_generic listing', 'event_lead',
       'event_search engine hit', 'event_searched products',
       'event_staticpage', 'event_viewed product', 'event_visited site']].multiply(time['time_score'], axis="index")

time.head()

,person,timestamp,hour,day,month,time_score,event_ad campaign hit,event_brand listing,event_checkout,event_conversion,event_generic listing,event_lead,event_search engine hit,event_searched products,event_staticpage,event_viewed product,event_visited site
0,4886f805,2018-05-18 00:11:59,0,18,5,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,0.0
1,ad93850f,2018-05-18 00:11:27,0,18,5,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,0.0
2,0297fc1e,2018-05-18 00:11:16,0,18,5,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,0.0
3,2d681dd8,2018-05-18 00:11:14,0,18,5,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,0.0
4,cccea85e,2018-05-18 00:11:09,0,18,5,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,0.0


In [30]:
tiempo = time.drop(['timestamp', 'time_score', 'hour', 'day', 'month'], axis=1).groupby('person').sum()


In [31]:
tiempo.head()

,event_ad campaign hit,event_brand listing,event_checkout,event_conversion,event_generic listing,event_lead,event_search engine hit,event_searched products,event_staticpage,event_viewed product,event_visited site
person,,,,,,,,,,,
0008ed71,0.0,0.0,25.5,0.0,8.5,0.0,0.0,0.0,0.0,0.0,17.0
00091926,160.0,187.0,23.0,0.0,0.0,0.0,0.0,0.0,0.0,3102.5,275.5
00091a7a,2.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,2.0
000ba417,8.5,239.5,62.5,13.0,148.5,0.0,8.5,0.0,0.0,1749.0,63.5
000c79fe,14.5,0.0,14.5,0.0,14.5,0.0,14.5,130.5,0.0,43.5,14.5


In [12]:
#time_grouped = time.groupby('person').agg({'time_score':'sum'}).reset_index()

In [37]:
df_basicos =pd.merge(df_basicos, tiempo, on='person', how='inner')

In [38]:
df_basicos=df_basicos.fillna(0)
df_basicos.to_csv('features_basicos.csv',index=False)

In [39]:
df_basicos.shape

(38829, 29)

In [40]:
df_basicos

,person,Direct,Email,Organic,Paid,Referral,Social,ad campaign hit,brand listing,checkout,...,event_brand listing,event_checkout,event_conversion,event_generic listing,event_lead,event_search engine hit,event_searched products,event_staticpage,event_viewed product,event_visited site
0,0008ed71,2.0,0.0,0.0,0.0,0.0,0.0,0,0,3,...,0.0,25.5,0.0,8.5,0.0,0.0,0.0,0.0,0.0,17.0
1,00091926,1.0,0.0,25.0,0.0,0.0,0.0,15,25,2,...,187.0,23.0,0.0,0.0,0.0,0.0,0.0,0.0,3102.5,275.5
2,00091a7a,0.0,0.0,0.0,0.0,0.0,0.0,1,5,0,...,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,2.0
3,000ba417,0.0,0.0,6.0,0.0,0.0,0.0,1,24,6,...,239.5,62.5,13.0,148.5,0.0,8.5,0.0,0.0,1749.0,63.5
4,000c79fe,0.0,0.0,0.0,0.0,0.0,0.0,1,0,1,...,0.0,14.5,0.0,14.5,0.0,14.5,130.5,0.0,43.5,14.5
5,000e4d9e,5.0,0.0,5.0,1.0,0.0,0.0,19,17,1,...,199.5,8.0,0.0,136.0,0.0,48.0,0.0,0.0,3526.0,154.5
6,000e619d,0.0,0.0,2.0,2.0,0.0,0.0,6,11,1,...,91.5,8.5,0.0,65.5,0.0,23.0,51.5,0.0,240.5,42.0
7,001001be,0.0,0.0,0.0,3.0,0.0,0.0,0,0,3,...,0.0,24.0,8.0,24.0,0.0,0.0,136.0,0.0,328.0,28.5
8,0010e89a,0.0,0.0,0.0,0.0,0.0,0.0,1,0,1,...,0.0,15.0,0.0,0.0,0.0,0.0,0.0,0.0,15.0,15.0
9,0016c4b5,0.0,0.0,0.0,0.0,0.0,0.0,1,0,1,...,0.0,14.5,0.0,0.0,0.0,0.0,0.0,0.0,14.5,14.5
